In [ ]:
# Initialize roberta-base
from openprompt.plms import get_model_class
model_class = get_model_class(plm_type = "roberta-base")
model_path = 'roberta-base'
config = model_class.config.from_pretrained(model_path)
tokenizer = model_class.tokenizer.from_pretrained(model_path)
model = model_class.model.from_pretrained(model_path)

In [ ]:
# Initialize prompt template
from openprompt.prompts import ManualTemplate
promptTemplate = ManualTemplate(
    text = ["<text_a>", "It", "was", "<mask>"],
    tokenizer = tokenizer,
)

In [ ]:
# Initialize verbalizer
from openprompt.prompts import ManualVerbalizer
classes = ["negative", "positive"]

promptVerbalizer = ManualVerbalizer(
    classes = classes,
    label_words = {
        "negative": ["bad"],
        "positive": ["great"],
    },
    tokenizer = tokenizer,
)

In [ ]:
# Create prompt model
from openprompt import PromptForClassification
promptModel = PromptForClassification(
    template = promptTemplate,
    model = model,
    verbalizer = promptVerbalizer,
)

In [ ]:
# Data Processor
import os
import pandas as pd

from openprompt.data_utils import InputExample
from openprompt.data_utils.data_processor import DataProcessor

class SST2Processor(DataProcessor):
    
    def __init__(self):
        super().__init__()
        self.labels = ["negative", "positive"]

    def get_examples(self, data_dir, split):
        examples = []
        path = os.path.join(data_dir, "{}.tsv".format(split))
        df = pd.read_csv(path, sep='\\t', header = 0)
        sentences = df['sentence']
        labels = df['label']
        for idx in range(len(sentences)):
            sentence, label = sentences[idx], labels[idx]
            example = InputExample(
                guid = idx, text_a = sentence, label = int(label))
            examples.append(example)
                
        return examples

In [ ]:
# Load dataset
path_to_train = './custom_data'
dataset = SST2Processor().get_examples(path_to_train, 'train')

In [ ]:
# Create Data Loader
from openprompt import PromptDataLoader
data_loader = PromptDataLoader(
    dataset = dataset,
    tokenizer = tokenizer,
    template = promptTemplate,
)

In [ ]:
from openprompt.trainer import ClassificationRunner
from openprompt.config import get_default_config
trainer = ClassificationRunner(model, train_dataloader = data_loader, config = get_default_config())
num_of_epochs = 10
trainer.train_epoch(num_of_epochs)